Ici se trouve le code correspondant à la partie batch du schéma d'architecture du cas d'usage d'implémentation de d'analyse data de cryptomonnaie. 

Il s'agit de récupérer des données, sur une plage de temps donnée, depuis une source (ici bigquery) puis de lancer une pipeline de données (locale pour les tests avec interactiverunner() puis sur le cloud grâce à dataflowrunner()). 

Le dataflow exporte toutes les données traitées dans une table de BigQuery. 
Ces données sont prêtes à être visualisé depuis un outil de visualisation de données au choix (DataStudio, Grafana, etc).

In [7]:
import logging
import re
import json
import time
import traceback
#from utils.utils import *
#from utils.solutions import solutions
import google.auth

from IPython.core.display import display, HTML

import apache_beam as beam
from apache_beam import FlatMap, Map, ParDo, Flatten, Filter
from apache_beam import Values, CombineGlobally, CombinePerKey
from apache_beam import pvalue, window, WindowInto
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.combiners import Top, Mean, Count
from apache_beam.io.textio import ReadFromText, WriteToText
from apache_beam.io.gcp.pubsub import ReadFromPubSub
from apache_beam.io.gcp.bigquery import BigQueryDisposition, WriteToBigQuery

from apache_beam.runners import DataflowRunner
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

from apache_beam.testing.util import assert_that, is_empty, equal_to

from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions

In [8]:
from google.cloud import bigquery
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [48]:
project = google.auth.default()[1]
project

'crypto-beta-test'

Créer la dataset où sera stocké la table résultant de l'exécution de notre pipeline batch

In [49]:
!bq mk --location US --dataset temp_crypto_batch

BigQuery error in mk operation: Dataset 'crypto-beta-test:temp_crypto_batch'
already exists.


Créer le bucket qui servira de stockage pendant l'execution de la pipeline. Reminder : le nom de ce bucket doit être unique.

In [21]:
%%bash

project=$(gcloud config get-value project 2> /dev/null)
echo gs://temp_crypto_batch_${project//crypto-beta-test/bucket-beta}
gsutil mb gs://temp_crypto_batch_${project//crypto-beta-test/bucket-beta}

gs://temp_crypto_batch_bucket-beta


Creating gs://temp_crypto_batch_bucket-beta/...
ServiceException: 409 A Cloud Storage bucket named 'temp_crypto_batch_bucket-beta' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


CalledProcessError: Command 'b'\nproject=$(gcloud config get-value project 2> /dev/null)\necho gs://temp_crypto_batch_${project//crypto-beta-test/bucket-beta}\ngsutil mb gs://temp_crypto_batch_${project//crypto-beta-test/bucket-beta}\n'' returned non-zero exit status 1.

Vérifier dans Cloud Storage que le bucket a effectivement été créé.

In [28]:
project = google.auth.default()[1]
bucket = "gs://temp_crypto_batch_bucket-beta/" + project.replace('-', '_')

options = PipelineOptions(
        project=project,
        staging_location="%s/staging_bq_notebook" % bucket,
        temp_location="%s/temp_bq_notebook" % bucket
)

Attention : Il faut venir modifier dans la requête suivante la période de temps qui nous intéresse. Attention à bien régler tous les paramètres si vous souhaitez lancer le dataflow sur toutes les données, cela peut-être très long (en l'occurence plus de 2h30). Pour notre essai, nous allons restreindre la période temporelle à étudier

In [29]:
query = '''
#standardSQL
-- parse requests for Aggregator script (oracle_script_id = 3)
WITH parsed_aggregator_oracle_requests AS (
    SELECT ARRAY(
        SELECT JSON_EXTRACT_SCALAR(symbol_as_json, '$')
        FROM UNNEST(JSON_EXTRACT_ARRAY(decoded_result.calldata, "$.symbols")) AS symbol_as_json
    ) AS symbols,
    CAST(JSON_EXTRACT_SCALAR(decoded_result.calldata, "$.multiplier") AS FLOAT64) AS multiplier,
    ARRAY(
        SELECT CAST(JSON_EXTRACT_SCALAR(rate_as_json, '$') AS FLOAT64)
        FROM UNNEST(JSON_EXTRACT_ARRAY(decoded_result.result, "$.rates")) AS rate_as_json
    ) AS rates,
    block_timestamp,
    block_timestamp_truncated,
    oracle_request_id,
    FROM `public-data-finance.crypto_band.oracle_requests`
    WHERE request.oracle_script_id = 3
      AND decoded_result.calldata IS NOT NULL
      AND decoded_result.result IS NOT NULL
),
-- zip symbols and rates
zipped_rates AS (
    SELECT block_timestamp,
        block_timestamp_truncated,
        oracle_request_id,
        struct(symbol, rates[OFFSET(off)] AS rate) AS zipped,
        multiplier,
    FROM parsed_aggregator_oracle_requests,
        UNNEST(symbols) AS symbol WITH OFFSET off
    WHERE ARRAY_LENGTH(symbols) = ARRAY_LENGTH(rates)
),
-- adjust for multiplier
adjusted_rates AS (
    SELECT 
        block_timestamp,
        block_timestamp_truncated,
        oracle_request_id,
        struct(zipped.symbol, CAST(IEEE_DIVIDE(zipped.rate, multiplier) AS STRING) AS rate) AS zipped, 
    FROM zipped_rates
)
SELECT 
    block_timestamp,
    block_timestamp_truncated,
    oracle_request_id,
    zipped.symbol,
    zipped.rate,
FROM adjusted_rates
WHERE zipped.symbol = 'ETH' AND block_timestamp > '2021-07-26T00:00:00.0000000000Z'
ORDER BY block_timestamp ASC
--LIMIT 1000 
'''

In [30]:
client = bigquery.Client() 
query_df = client.query(query).result().to_dataframe()
query_df

/root/apache-beam-2.25.0/lib/python3.7/site-packages/google/cloud/bigquery/client.py:407: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,block_timestamp,block_timestamp_truncated,oracle_request_id,symbol,rate
0,2021-07-26T00:00:04.807849815Z,2021-07-26 00:00:04.807849+00:00,8121963,ETH,2191.679999999
1,2021-07-26T00:00:11.650717545Z,2021-07-26 00:00:11.650717+00:00,8121968,ETH,2191.9533
2,2021-07-26T00:00:17.223110841Z,2021-07-26 00:00:17.223110+00:00,8121973,ETH,2192
3,2021-07-26T00:00:30.108639482Z,2021-07-26 00:00:30.108639+00:00,8121978,ETH,2192
4,2021-07-26T00:00:44.733576216Z,2021-07-26 00:00:44.733576+00:00,8121986,ETH,2191.94
...,...,...,...,...,...
97991,2021-08-06T23:59:14.750372181Z,2021-08-06 23:59:14.750372+00:00,8717431,ETH,2893.19
97992,2021-08-06T23:59:27.237776938Z,2021-08-06 23:59:27.237776+00:00,8717435,ETH,2893.19
97993,2021-08-06T23:59:36.667968309Z,2021-08-06 23:59:36.667968+00:00,8717441,ETH,2892.983
97994,2021-08-06T23:59:50.644331018Z,2021-08-06 23:59:50.644331+00:00,8717448,ETH,2892.983


2. Interactive Runner

Testons la pipeline de données en local avec interactiverunner(). Le code va donc se lancer et s'executer depuis la VM. 

In [31]:
logging.getLogger().setLevel(logging.WARNING)

In [32]:
import pandas 

class KeepDoFn(beam.DoFn):
    def process(self, element):
        return [{
            'timestamp': element['block_timestamp_truncated'],
            'request': str(element['oracle_request_id']),
            'symbol' : element['symbol'],
            'rate' : element['rate'], 
            'date_str' : pandas.to_datetime(element['block_timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ').strftime("%Y%m%d%H%M%S")
    }]
    
table = "{}:temp_crypto_batch.batch_1".format(project)
schema = "timestamp:TIMESTAMP, request:STRING, symbol:STRING, rate:FLOAT, date_str:STRING"

p = beam.Pipeline(InteractiveRunner(), options=options)
create = (p | "Create" >>  beam.io.ReadFromBigQuery(query=query,use_standard_sql=True)
            | "Map Keys" >> beam.ParDo(KeepDoFn())
            | "Write To BigQuery" >> WriteToBigQuery(table=table, schema=schema,
                                  create_disposition=BigQueryDisposition.CREATE_IF_NEEDED,
                                  write_disposition=BigQueryDisposition.WRITE_APPEND))

ib.show_graph(p)

In [33]:
ib.show(create)

On peut visualiser que l'insertion s'est effectuée correctement en effectuant une requête à bigquery depuis ce notebook

In [34]:
query = '''
SELECT *  FROM `{project}.temp_crypto_batch.batch_1` LIMIT 10
'''.format(project=project)

In [35]:
query

'\nSELECT *  FROM `crypto-beta-test.temp_crypto_batch.batch_1` LIMIT 10\n'

In [36]:
client = bigquery.Client() 
query_df = client.query(query).result().to_dataframe()
query_df

/root/apache-beam-2.25.0/lib/python3.7/site-packages/google/cloud/bigquery/client.py:407: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,timestamp,request,symbol,rate,date_str
0,2021-07-26 01:48:06.037118+00:00,8125734,ETH,2304.0,20210726014806
1,2021-07-26 01:48:37.149982+00:00,8125749,ETH,2304.0,20210726014837
2,2021-07-26 01:49:23.049240+00:00,8125773,ETH,2304.0,20210726014923
3,2021-07-26 02:17:46.696775+00:00,8126766,ETH,2304.0,20210726021746
4,2021-07-26 02:17:53.665170+00:00,8126771,ETH,2304.0,20210726021753
5,2021-07-27 13:56:32.651279+00:00,8199583,ETH,2304.0,20210727135632
6,2021-07-27 14:01:13.594706+00:00,8199740,ETH,2304.0,20210727140113
7,2021-07-28 10:21:05.861121+00:00,8238839,ETH,2304.0,20210728102105
8,2021-07-28 10:21:44.232029+00:00,8238862,ETH,2304.0,20210728102144
9,2021-07-28 16:12:59.640273+00:00,8250572,ETH,2304.0,20210728161259


3. DataflowRunner

Cette fois-ci, testons le dataflow directement sur le cloud avec dataflowrunner(). Le code va donc se lancer depuis la VM et s'executer sur Dataflow.

In [37]:
logging.getLogger().setLevel(logging.INFO)

In [38]:
from setuptools import setup, find_packages

In [39]:
%%writefile setup.py

from setuptools import setup, find_packages

setup(
    name="dataflow_pipeline_dependencies",
    version="0.1",
    install_requires=[
   'pandas==0.25.2',
    ],
    packages = find_packages()
)

Overwriting setup.py


In [40]:
query = '''
#standardSQL
-- parse requests for Aggregator script (oracle_script_id = 3)
WITH parsed_aggregator_oracle_requests AS (
    SELECT ARRAY(
        SELECT JSON_EXTRACT_SCALAR(symbol_as_json, '$')
        FROM UNNEST(JSON_EXTRACT_ARRAY(decoded_result.calldata, "$.symbols")) AS symbol_as_json
    ) AS symbols,
    CAST(JSON_EXTRACT_SCALAR(decoded_result.calldata, "$.multiplier") AS FLOAT64) AS multiplier,
    ARRAY(
        SELECT CAST(JSON_EXTRACT_SCALAR(rate_as_json, '$') AS FLOAT64)
        FROM UNNEST(JSON_EXTRACT_ARRAY(decoded_result.result, "$.rates")) AS rate_as_json
    ) AS rates,
    block_timestamp,
    block_timestamp_truncated,
    oracle_request_id,
    FROM `public-data-finance.crypto_band.oracle_requests`
    WHERE request.oracle_script_id = 3
      AND decoded_result.calldata IS NOT NULL
      AND decoded_result.result IS NOT NULL
),
-- zip symbols and rates
zipped_rates AS (
    SELECT block_timestamp,
        block_timestamp_truncated,
        oracle_request_id,
        struct(symbol, rates[OFFSET(off)] AS rate) AS zipped,
        multiplier,
    FROM parsed_aggregator_oracle_requests,
        UNNEST(symbols) AS symbol WITH OFFSET off
    WHERE ARRAY_LENGTH(symbols) = ARRAY_LENGTH(rates)
),
-- adjust for multiplier
adjusted_rates AS (
    SELECT 
        block_timestamp,
        block_timestamp_truncated,
        oracle_request_id,
        struct(zipped.symbol, CAST(IEEE_DIVIDE(zipped.rate, multiplier) AS STRING) AS rate) AS zipped, 
    FROM zipped_rates
)
SELECT 
    block_timestamp,
    block_timestamp_truncated,
    oracle_request_id,
    zipped.symbol,
    zipped.rate,
FROM adjusted_rates
--WHERE block_timestamp between '2021-05-20T00:00:00.000000000Z' AND '2021-06-01T00:00:00.000000000Z'
WHERE block_timestamp < '2021-05-01T00:00:00.0000000000Z'
ORDER BY block_timestamp DESC
--LIMIT 1000 
'''

In [41]:
pipeline_parameters = [
    '--project', project,
    '--staging_location', "%s/staging_bq_notebook" % bucket,
    '--temp_location', "%s/temp_bq_notebook" % bucket,
    "--setup_file", './setup.py', 
    "--region", "us-central1",
]

Activez l'API permettant d'executer des pipelines de données directement sur Dataflow (si cela n'est pas déjà fait).

In [42]:
!gcloud services enable dataflow.googleapis.com 

In [43]:
class KeepDoFn(beam.DoFn):
    def process(self, element):
        import pandas 
        
        return [{
            'timestamp': element['block_timestamp_truncated'],
            'request': str(element['oracle_request_id']),
            'symbol' : element['symbol'],
            'rate' : element['rate'], 
            'date_str' : pandas.to_datetime(element['block_timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ').strftime("%Y%m%d%H%M%S")
    }]
    
table = "{}:temp_crypto_batch.batch_timestamp".format(project)
schema = "timestamp:TIMESTAMP, request:STRING, symbol:STRING, rate:FLOAT, date_str:STRING"

p = beam.Pipeline("DataFlowRunner", argv=pipeline_parameters)
create = (p | "Create" >>  beam.io.ReadFromBigQuery(query=query,use_standard_sql=True)
            | "Map Keys" >> beam.ParDo(KeepDoFn())
            | "Write To BigQuery" >> WriteToBigQuery(table=table, schema=schema,
                                  create_disposition=BigQueryDisposition.CREATE_IF_NEEDED,
                                  write_disposition=BigQueryDisposition.WRITE_APPEND))

p.run()

INFO:apache_beam.runners.portability.stager:Executing command: ['/root/apache-beam-2.25.0/bin/python', 'setup.py', 'sdist', '--dist-dir', '/tmp/tmpish3jm3o']
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/root/apache-beam-2.25.0/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmpish3jm3o', 'apache-beam==2.25.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI: dataflow_python_sdk.tar
INFO:apache_beam.runners.portability.stager:Downloading binary distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/root/apache-beam-2.25.0/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmpish3jm3o', 'apache-beam==2.25.0', '--no-deps', '--only-binary', ':all:', '--python-version', '37', '--implementation', 'cp', '--abi', 'cp37m', '--platform', 'manylinux1_x86_64']
INFO:a

<DataflowPipelineResult <Job
 createTime: '2021-08-07T08:17:24.821572Z'
 currentStateTime: '1970-01-01T00:00:00Z'
 id: '2021-08-07_01_17_23-14499271834719554261'
 location: 'us-central1'
 name: 'beamapp-root-0807081720-792216'
 projectId: 'crypto-beta-test'
 stageStates: []
 startTime: '2021-08-07T08:17:24.821572Z'
 steps: []
 tempFiles: []
 type: TypeValueValuesEnum(JOB_TYPE_BATCH, 1)> at 0x7f8fca096ed0>

Dans les warning ci-dessus, on peut visualiser directement l'execution de la pipeline de données sur dataflow. <br>
NB : Si la première execution échoue (création d'un nouveau compte), relancer le dataflow une deuxième fois (cela est surement dû à un retard de mise à jour des droits), cela devrait fonctionner la deuxième fois (car nous avons activé l'API de Dataflow précédemment)

Une fois l'exécution terminée, de la même manière qu'avec interactive runner en locale sur la VM, on peut visualiser que l'insertion s'est bien effectuée correctement en effectuant une requête à bigquery depuis ce notebook

In [46]:
query = '''
SELECT *  FROM `temp_crypto_batch.batch_timestamp` LIMIT 10
'''

In [47]:
client = bigquery.Client() 
query_df = client.query(query).result().to_dataframe()
query_df

/root/apache-beam-2.25.0/lib/python3.7/site-packages/google/cloud/bigquery/client.py:407: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,timestamp,request,symbol,rate,date_str
0,2021-04-24 21:39:20.187837+00:00,4153686,ETH,2286.808050,20210424213920
1,2021-04-24 21:39:02.986974+00:00,4153676,ETH,2286.435400,20210424213902
2,2021-04-24 21:38:33.330373+00:00,4153655,BNB,508.331150,20210424213833
3,2021-04-24 21:38:19.663021+00:00,4153651,ATOM,18.990000,20210424213819
4,2021-04-24 21:37:28.756977+00:00,4153621,SUSD,1.020000,20210424213728
5,2021-04-24 21:36:03.333411+00:00,4153575,AAVE,327.425500,20210424213603
6,2021-04-24 21:36:03.333411+00:00,4153575,ZRX,1.361614,20210424213603
7,2021-04-24 21:36:00.657166+00:00,4153573,OGN,1.520000,20210424213600
8,2021-04-24 21:34:57.208747+00:00,4153538,LINK,32.376400,20210424213457
9,2021-04-24 21:34:52.539335+00:00,4153535,OMG,6.210000,20210424213452


Cette étape terminée, il est possible :
- De visualiser ces données à travers datastudio 
- De s'occuper de la partie streaming de l'architecture
- D'identifier les valeurs manquantes ou aberrantes et d'effectuer de l'interpolation pour préparer les données à être utilisé dans un modèle d'apprentissage dans le but d'effectuer de la prédiction ou de la détection d'anomalies